<a href="https://colab.research.google.com/github/xutian1113/pytorch_practice/blob/main/glue_CoLA_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install datasets
!pip install tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_syst

## CoLA Description
CoLA (Corpus of Linguistic Acceptability) is a dataset used for evaluating a model's ability to determine whether a given sentence is gramatically acceptable or unacceptable.
- Task Type: binary classification (Acceptable or Unacceptable)
- Goal: Predict whether a given sentence is grammatically acceptable (1) or unacceptable (0).
- Data Source: Sentences are taken from published linguistic literature.
- Label Distribution:
  - 1 (Acceptable Sentence) → The sentence follows standard English grammar.
  - 0 (Unacceptable Sentence) → The sentence is grammatically incorrect.
  
### CoLA Dataset Statistics
| **Split**      | **Number of Samples** |
|---------------|----------------------|
| **Train**     | 8,551                |
| **Validation** | 1,043                |
| **Test**      | ~1,000 (Labels not public) |



```
DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 8551
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1043
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1063
    })
})
```

### dataset["train"][0]
```
{'sentence': "Our friends won't buy this analysis, let alone the next one we propose.",
 'label': 1,
 'idx': 0}
```

### dataset statistics
- train: ({0: 2528, 1: 6023}, 0.704362062916618)
- val: ({(0: 322, 1: 721}, 0.6912751677852349)
- test: unavailable


In [4]:
import torch
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from datasets import load_dataset
import numpy as np

from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import confusion_matrix

from tqdm import tqdm


In [5]:
def accuracy(preds, labels):
  _, predicted = torch.max(preds, 1)
  correct = (predicted == labels).sum().item()
  return correct / len(labels)

def f1_score(preds, labels):
  _, predicted = torch.max(preds, 1)
  tp = ((predicted == 1) & (labels == 1)).sum().item()
  fp = ((predicted == 1) & (labels == 0)).sum().item()
  fn = ((predicted == 0) & (labels == 1)).sum().item()
  precision = tp / (tp + fp) if tp + fp > 0 else 0
  recall = tp / (tp + fn) if tp + fn > 0 else 0
  f1 = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0
  return f1

def precision(preds, labels):
  _, predicted = torch.max(preds, 1)
  tp = ((predicted == 1) & (labels == 1)).sum().item()
  fp = ((predicted == 1) & (labels == 0)).sum().item()
  precision = tp / (tp + fp) if tp + fp > 0 else 0
  return precision


def auc_roc(preds, labels):
  # Convert predictions and labels to NumPy arrays
  _, predicted = torch.max(preds, 1)
  preds_np = predicted.detach().cpu().numpy()
  labels_np = labels.detach().cpu().numpy()

  # Calculate AUC-ROC using scikit-learn's roc_auc_score
  auc_score = roc_auc_score(labels_np, preds_np)

  return auc_score


def auc_pr(preds, labels):
  """
  Calculates the AUPRC score for binary classification.

  Args:
    preds: Predicted probabilities (tensor) for the positive class.
    labels: True labels (tensor) (0 or 1).

  Returns:
    AUPRC score (float).
  """
  # Convert predictions and labels to NumPy arrays
  _, predicted = torch.max(preds, 1)
  preds_np = predicted.detach().cpu().numpy()
  labels_np = labels.detach().cpu().numpy()

  # Calculate AUPRC using scikit-learn's average_precision_score
  auprc_score = average_precision_score(labels_np, preds_np)

  return auprc_score

def confusion_matrix_func(preds, labels, num_classes=2):
  """
  Calculates the confusion matrix.

  Args:
    preds: Predicted labels (tensor).
    labels: True labels (tensor).
    num_classes: Number of classes. Defaults to 2 for binary classification.

  Returns:
    Confusion matrix (NumPy array).
  """
  # Get predicted class indices
  _, predicted = torch.max(preds, 1)

  # Convert predictions and labels to NumPy arrays
  predicted_np = predicted.detach().cpu().numpy()
  labels_np = labels.detach().cpu().numpy()

  # Calculate confusion matrix using scikit-learn's confusion_matrix
  cm = confusion_matrix(labels_np, predicted_np, labels=range(num_classes))

  return cm

def recall(preds, labels):
  """
  Calculates the recall score for binary classification.

  Args:
    preds: Predicted labels (tensor).
    labels: True labels (tensor).

  Returns:
    Recall score (float).
  """
  # Get predicted class indices
  _, predicted = torch.max(preds, 1)

  # Calculate true positives (TP), false negatives (FN)
  tp = ((predicted == 1) & (labels == 1)).sum().item()
  fn = ((predicted == 0) & (labels == 1)).sum().item()

  # Calculate recall
  recall_score = tp / (tp + fn) if tp + fn > 0 else 0  # Avoid division by zero

  return recall_score



In [18]:
# 1. load the GLUE MRPC dataset
dataset = load_dataset('glue', 'cola')

# 2. load the tokenizer for bert-base-cased
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
# converting raw text into numerical representations that can be processed by BERT model.

# 3. define a tokenization function for sentence pair

def tokenize_function(example):
    return tokenizer(example["sentence"], truncation=True, padding="max_length", max_length=128)

'''
input_ids: Tokenized numerical representation of both sentences.
token_type_ids: Differentiates sentence1 (0) from sentence2 (1).
attention_mask: Indicates which tokens are real (1) and padding (0).
'''
# 4. Tokenize the dataset (batched processing)
tokenized_datasets = dataset.map(tokenize_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Map:   0%|          | 0/8551 [00:00<?, ? examples/s]

Map:   0%|          | 0/1043 [00:00<?, ? examples/s]

Map:   0%|          | 0/1063 [00:00<?, ? examples/s]

In [24]:
# 5. Remove unnecessary columns and set the format to PyTorch tensors
# Remove columns that are not used by the model (like the original sentences and index)
tokenized_datasets = tokenized_datasets.remove_columns(["sentence", "idx"])
tokenized_datasets.set_format("torch")

# 6. Create DataLoaders for training and validation
train_dataloader = DataLoader(tokenized_datasets["train"], shuffle=True, batch_size=8)
val_dataloader = DataLoader(tokenized_datasets["validation"], batch_size=8)


# 7. Set up the device, model, optimizer, and scheduler
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BertForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)
model.to(device)

# optimizer = AdamW(model.parameters(), lr=2e-5)
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
epochs = 20
total_steps = len(train_dataloader) * epochs

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)



model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# # 1. load the GLUE MRPC dataset
# dataset = load_dataset('glue', 'cola')

# # 2. load the tokenizer for bert-base-cased
# tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
# # converting raw text into numerical representations that can be processed by BERT model.

# # 3. define a tokenization function for sentence pair

# def tokenize_function(example):
#     return tokenizer(example["sentence"], truncation=True, padding="max_length", max_length=128)

# '''
# input_ids: Tokenized numerical representation of both sentences.
# token_type_ids: Differentiates sentence1 (0) from sentence2 (1).
# attention_mask: Indicates which tokens are real (1) and padding (0).
# '''
# # 4. Tokenize the dataset (batched processing)
# tokenized_datasets = dataset.map(tokenize_function, batched=True)

# # 5. Remove unnecessary columns and set the format to PyTorch tensors
# # Remove columns that are not used by the model (like the original sentences and index)
# tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "idx"])
# tokenized_datasets.set_format("torch")

# # 6. Create DataLoaders for training and validation
# train_dataloader = DataLoader(tokenized_datasets["train"], shuffle=True, batch_size=8)
# val_dataloader = DataLoader(tokenized_datasets["validation"], batch_size=8)


# # 7. Set up the device, model, optimizer, and scheduler
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = BertForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)
# model.to(device)

# # optimizer = AdamW(model.parameters(), lr=2e-5)
# optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
# epochs = 20
# total_steps = len(train_dataloader) * epochs

# scheduler = get_linear_schedule_with_warmup(
#     optimizer,
#     num_warmup_steps=0,
#     num_training_steps=total_steps
# )


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
best_val_loss = float('inf')
best_model_state = None
best_epoch = 0
epochs_no_improve = 0
patience = 3

model.train()
for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    total_loss = 0
    all_preds = []
    all_labels = []
    for batch in tqdm(train_dataloader, desc=f"Training Epoch {epoch + 1}"):
        # Move batch to the device
        batch = {key: value.to(device) for key, value in batch.items()}
        batch['labels'] = batch['label']
        del batch['label']
        # Forward pass
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.item()

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()
        # predictions = torch.argmax(outputs.logits, dim=-1)
        all_preds.append(outputs.logits.detach())
        all_labels.append(batch['labels'].detach())

    all_preds = torch.cat(all_preds)
    all_labels = torch.cat(all_labels)

    train_acc = accuracy(all_preds, all_labels)
    train_auc = auc_roc(all_preds, all_labels)
    train_rec = recall(all_preds, all_labels)
    train_avg_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch+1}/{epochs} - Loss: {train_avg_loss:.4f}, Accuracy: {train_acc:.4f}, AUC-ROC: {train_auc:.4f}, Recall: {train_rec:.4f}")

    # 9. Validation loop
    model.eval()
    correct_predictions = 0
    total_predictions = 0
    val_preds = []
    val_labels = []
    val_loss = 0
    for batch in val_dataloader:
        batch = {key: value.to(device) for key, value in batch.items()}
        batch['labels'] = batch['label']
        del batch['label']
        with torch.no_grad():
            outputs = model(**batch)
            val_loss += outputs.loss.item()
        logits = outputs.logits
        val_preds.append(logits.detach())
        val_labels.append(batch['labels'].detach())

    val_preds = torch.cat(val_preds)
    val_labels = torch.cat(val_labels)
    val_acc = accuracy(val_preds, val_labels)
    val_auc = auc_roc(val_preds, val_labels)
    val_rec = recall(val_preds, val_labels)
    val_avg_loss = val_loss / len(val_dataloader)
    if val_avg_loss < best_val_loss:
        best_val_loss = val_avg_loss
        best_model_state = model.state_dict()
        best_epoch = epoch
        epochs_no_improve = 0
    else:
      epochs_no_improve += 1
      if epochs_no_improve >= patience:
          print(f"Early stopping at epoch {epoch+1} due to no improvement in validation loss for {patience} epochs.")
          break
    print(f"Validation - Loss: {val_avg_loss:.4f},  Accuracy: {val_acc:.4f}, AUC-ROC: {val_auc:.4f}, Recall: {val_rec:.4f}")


    model.train()  # Switch back to training mode for the next epoch


Epoch 1/20


Training Epoch 1: 100%|██████████| 1069/1069 [00:52<00:00, 20.45it/s]


Epoch 1/20 - Loss: 0.4918, Accuracy: 0.7644, AUC-ROC: 0.6827, Recall: 0.8825
Validation - Accuracy: 0.8188, AUC-ROC: 0.7521, Recall: 0.9265
Epoch 2/20


Training Epoch 2: 100%|██████████| 1069/1069 [00:51<00:00, 20.87it/s]


Epoch 2/20 - Loss: 0.2830, Accuracy: 0.8887, AUC-ROC: 0.8591, Recall: 0.9314
Validation - Accuracy: 0.8313, AUC-ROC: 0.7645, Recall: 0.9390
Epoch 3/20


Training Epoch 3: 100%|██████████| 1069/1069 [00:51<00:00, 20.87it/s]


Epoch 3/20 - Loss: 0.1536, Accuracy: 0.9426, AUC-ROC: 0.9308, Recall: 0.9597
Validation - Accuracy: 0.8150, AUC-ROC: 0.7373, Recall: 0.9404
Epoch 4/20


Training Epoch 4:  34%|███▍      | 363/1069 [00:17<00:33, 20.85it/s]

In [ ]:
test_dataloader = DataLoader(tokenized_datasets["test"], batch_size=8)
model.eval()
test_preds = []
test_labels = []

for batch in test_dataloader:
    batch = {key: value.to(device) for key, value in batch.items()}
    batch['labels'] = batch['label']
    del batch['label']
    with torch.no_grad():
        outputs = model(**batch)
    logits = outputs.logits
    test_preds.append(logits.detach())
    test_labels.append(batch['labels'].detach())

test_preds = torch.cat(test_preds)
test_labels = torch.cat(test_labels)

test_acc = accuracy(test_preds, test_labels)
test_auc = auc_roc(test_preds, test_labels)
test_rec = recall(test_preds, test_labels)
print(f"Test - Accuracy: {test_acc:.4f}, AUC-ROC: {test_auc:.4f}, Recall: {test_rec:.4f}")